# <Center> **FLUJO DE LIMPIEZA, TRANSFORMACIÓN DE DATOS Y APLICACIÓN DEL MODELO** </Center> 

In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import LabelEncoder
import subprocess

---
---
# <Center> **DESCARGAR EL ARCHIVO A LA ZONA RAW LOCAL DESDE S3 (INGEST)** </Center> 

In [2]:
#Este paso se ejecuta a través de shell en donde se hace una descarga directa desde el bucket de S3 a través de IP's permitidas para acceder al bucket (Revisar documentación)
cmd = "aws s3 cp s3://taxdatalake/RAW/DCS_Item_Extract_Report C:\\Users\\user\\OneDrive\\EAFIT\\MAESTRÍA_EN_CIENCIA_DE_DATOS\\PROYECTO_INTEGRADOR\\BUCKET_PROJECT\\1_RAW\\ --no-sign-request"

# Correr el comando
subprocess.run(cmd, shell=True)

CompletedProcess(args='aws s3 cp s3://taxdatalake/RAW/DCS_Item_Extract_Report C:\\Users\\user\\OneDrive\\EAFIT\\MAESTRÍA_EN_CIENCIA_DE_DATOS\\PROYECTO_INTEGRADOR\\BUCKET_PROJECT\\1_RAW\\ --no-sign-request', returncode=0)

In [3]:
#Defino la ruta en donde se almacenó el archivo de S3 para recuperar los datos.
path = r'C:\\Users\\user\\OneDrive\\EAFIT\\MAESTRÍA_EN_CIENCIA_DE_DATOS\\PROYECTO_INTEGRADOR\\BUCKET_PROJECT\\1_RAW\\DCS_Item_Extract_Report'

In [4]:
#Función para realizar la limpieza de los datos desde la ruta definida

def cleaning(path):
    df = pd.read_excel(path,header = 1)
    df = df.iloc[0:, 1:]
    columns = df.iloc[0: ,2:20].columns
    for column in columns:
        df = df[df[column].notnull()]
    return(df)

df = cleaning(path)

# <CENTER> **EXPORTAR EL DATAFRAME A LA ZONA TRUSTED** </CENTER>

---
---

In [5]:
#Indico cuáles son las observaciones que quiero predecir a partir de mi conjunto de datos (Solo VERTEX PROD ID vacías)
target = df[df['VERTEX PROD ID'].isnull()]
target.fillna('MISS', inplace=True)
target['VERTEX PROD ID'] = target['VERTEX PROD ID'].replace('MISS', '')

C:\Users\user\AppData\Local\Temp\ipykernel_2748\1125565649.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target.fillna('MISS', inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_2748\1125565649.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target['VERTEX PROD ID'] = target['VERTEX PROD ID'].replace('MISS', '')


In [7]:
target.to_csv(r'C:\Users\user\OneDrive\EAFIT\MAESTRÍA_EN_CIENCIA_DE_DATOS\PROYECTO_INTEGRADOR\BUCKET_PROJECT\2_TRUSTED\DataFrame.csv', index_label='Index')

# <CENTER> **APLICAR EL LABEL ENCODER** </CENTER>

---
---

In [8]:
LabelEncode = joblib.load('ordinal_encoders.joblib')

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator OrdinalEncoder from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
def encode(df, encoding_table):
    encoded_df = df.copy()

    for col, encoder in encoding_table.items():
        if col in encoded_df.columns:  # Check if the column exists in the DataFrame
            # Handle NaN values before transforming
            encoded_df[col].fillna('MISS', inplace=True)  # Adjust 'missing' as needed

            # Transform the column using the encoder
            encoded_df[col] = encoder.transform(encoded_df[[col]]).flatten()

    return encoded_df

In [10]:
target_noitem = target.iloc[:,1:]

# Use the encode function to transform your data
encoded_df = encode(target_noitem, LabelEncode)

A partir de este punto, debemos transformar los datos a cada una de sus representaciones numéricas.

In [11]:
encoded_df = encoded_df.drop('VERTEX PROD ID',axis=1)
encoded_df.tail(1)

,DOM RLS CODE,MANUFACT WHSE,PRODUCT CLASS,PROD CLASS DESC,ITEM HSE PROD LINE,ITEM HSE PROD LINE DESC,BUSINESS SEGMENT CDE,BUSINESS SEGMENT DESC,APPLICATION CDE,APPLICATION DESC,...,PRODUCT GROUP,PRODUCT GROUP DESC,PROD SYSTEM,PROD SYSTEM DESC,PROD SUBSYSTEM,PROD SUBSYSTEM DESC,PROD FAMILY,PROD FAMILY DESC,MANUFACT TYPE CODE,INSTRUMENT DESC
63,16.0,10.0,43.0,35.0,3.0,4.0,2.0,8.0,20.0,25.0,...,11.0,29.0,263.0,213.0,207.0,357.0,3045.0,1682.0,2.0,4.0


# <CENTER> **EXTRAER LOS MODELOS** </CENTER>

In [12]:
#model = joblib.load(r"trained_model_rf.pkl")
#model = joblib.load(r'trained_model_XG.pkl')
#model = joblib.load(r'trained_model_Bagging.pkl')
model = joblib.load(r'Final_model.pkl')

c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator BaggingClassifier from version 1.3.0 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [13]:
#encoded_df['VERTEX PROD ID'] = random_forest_model.predict(encoded_df2)
encoded_df['VERTEX PROD ID'] = model.predict(encoded_df)

In [14]:
def decode(df, encoding_table):
    columns = df.columns
    for col in columns:
        df.loc[:, col] = encoding_table[col].inverse_transform(df[col].values.reshape(-1, 1))
    return df

In [15]:
encoded_df.head(3)

,DOM RLS CODE,MANUFACT WHSE,PRODUCT CLASS,PROD CLASS DESC,ITEM HSE PROD LINE,ITEM HSE PROD LINE DESC,BUSINESS SEGMENT CDE,BUSINESS SEGMENT DESC,APPLICATION CDE,APPLICATION DESC,...,PRODUCT GROUP DESC,PROD SYSTEM,PROD SYSTEM DESC,PROD SUBSYSTEM,PROD SUBSYSTEM DESC,PROD FAMILY,PROD FAMILY DESC,MANUFACT TYPE CODE,INSTRUMENT DESC,VERTEX PROD ID
0,4.0,2.0,9.0,61.0,9.0,3.0,8.0,17.0,30.0,57.0,...,113.0,392.0,309.0,552.0,496.0,3164.0,599.0,22.0,6.0,8
1,10.0,0.0,0.0,75.0,15.0,1.0,2.0,8.0,55.0,52.0,...,17.0,81.0,299.0,554.0,515.0,2730.0,2752.0,3.0,3.0,31
2,10.0,0.0,36.0,46.0,7.0,8.0,6.0,6.0,34.0,46.0,...,28.0,141.0,52.0,658.0,82.0,2410.0,622.0,3.0,3.0,1


# <CENTER> **HACER PREDICCIONES** </CENTER>

---
---

In [17]:
predicted_y = decode(encoded_df,LabelEncode)

In [22]:
predictions = pd.merge(target.iloc[:, :1], predicted_y, left_index=True, right_index=True)

In [26]:
#Separo las X no identificadas y que necesitan más información puesto que el modelo aún no las conoce.
New_features = predictions[predictions.isna().any(axis=1)]

In [27]:
#Me quedo solo con las predicciones que tienen datos completos y que serán clasificadas
predictions = predictions[predictions.notna().all(axis=1)]

# <CENTER> **EXPORTO MIS RESULTADOS A LA ZONA REFINED** </CENTER>

In [28]:
New_features.to_csv(r'C:\Users\user\OneDrive\EAFIT\MAESTRÍA_EN_CIENCIA_DE_DATOS\PROYECTO_INTEGRADOR\BUCKET_PROJECT\3_REFINED\New_features.csv', index_label='Index')

In [29]:
predictions.to_csv(r'C:\Users\user\OneDrive\EAFIT\MAESTRÍA_EN_CIENCIA_DE_DATOS\PROYECTO_INTEGRADOR\BUCKET_PROJECT\3_REFINED\Predictions.csv', index_label='Index')